In [65]:
# Wichtig!
#pip install opencv-python==4.5.4.60
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr
import pandas as pd
from os import listdir
from os.path import isfile, join
import pytesseract
from PIL import Image
import traceback
import sys
import fuzzywuzzy

In [2]:
#img = cv2.imread(r"D:\5_Integrationsseminar\Aufnahmen\still92.jpg")
img = cv2.imread(r"./data/small/KZE_008.jpg")
dir=r"./data/small"
dir_cropped=r"./data/cropped"
images = [f for f in listdir(dir) if isfile(join(dir, f))]

#C:/Users/Kilian/Notebook/5_Semster/test_plates_kaggle/IMG_4134.jpg
#gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))

In [51]:
# initializing Dataframe
df = pd.DataFrame(columns=['Image', 'Cropping', 'Text_easyocr','Text_pytesseract', 'score', 'score2'])
df.Image=images

labels = pd.read_csv("./data/labels.csv", sep=";")
df = df.merge(labels, on="Image")

In [52]:
df

,Image,Cropping,Text_easyocr,Text_pytesseract,Text_label
0,KZE_001.jpg,NaN,NaN,NaN,SG GE 567
1,KZE_002.jpg,NaN,NaN,NaN,K BA 4773
2,KZE_003.jpg,NaN,NaN,NaN,MA OV 7575
3,KZE_004.jpg,NaN,NaN,NaN,MA A 114
4,KZE_005.jpg,NaN,NaN,NaN,MA PP 444E
...,...,...,...,...,...
113,KZE_114.jpg,NaN,NaN,NaN,MA AI 2911
114,KZE_115.jpg,NaN,NaN,NaN,LU SR 171
115,KZE_116.jpg,NaN,NaN,NaN,MA EE 1923
116,KZE_117.jpg,NaN,NaN,NaN,MA XD 668


In [53]:
easyocr_reader = easyocr.Reader(['en'])
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\misarasa\AppData\Local\Programs\Tesseract-OCR\tesseract.exe" # Please adjust this

for n, i in enumerate(images):
    path=(r"./data/small/"+i)
    print(path)
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction mit geringem Einfluss auf die Kanten
    edged = cv2.Canny(bfilter, 30, 100) # Kantenerkennung
    #plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))
    #plt.show()
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break

    #print(location)

    mask = np.zeros(gray.shape, np.uint8)

    try:
        new_image = cv2.drawContours(mask, [location], 0,255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)
        #ab hier nummern
        #plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))
        #plt.show()
        

        (x,y) = np.where(mask==255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        # cropped_image = gray[x1:x2+1, y1:y2+1] # cOdEqUalItÄt
        cropped_image = new_image[x1:x2+1, y1:y2+1]
        df.iloc[n,1]=("Okay")
        cv2.imwrite((r"./data/cropped/"+i+'_cropped.jpg'),cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
        
        try: # ------- EasyOCR -------
            easyocr_result = easyocr_reader.readtext(cropped_image, min_size = 40, allowlist = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789ÜÖA")
            # print('EasyOCR Full Result: ',result)
            easyocr_text=''
            for x in easyocr_result:
                easyocr_text=easyocr_text+x[1]+" "
            print("easyOCR Result: ", easyocr_text)
            df.iloc[n,2] = easyocr_text
            
        except:
            print("easyOCR Fehler")
            df.iloc[n,2] = "Fehler_EasyOCR"
            print(traceback.format_exc())
         
        try: # ------- Tesseract -------
            tesseract_result = pytesseract.image_to_string(cropped_image, lang='eng', config='--psm 6')
            print("Tesseract Result: ", tesseract_result)
            df.iloc[n,3] = tesseract_result
        except:
            print("Tesseract Fehler")
            df.iloc[n,3] = "Fehler_Tesseract"

        
    except:
        print("Fehler Crop bei Bild", path)
        df.iloc[n,1]="Fehler"
    print("\n----------------------------------------------------------")
        
    

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


./data/small/KZE_001.jpg
easyOCR Result:  
Tesseract Result:  oe


----------------------------------------------------------
./data/small/KZE_002.jpg
easyOCR Result:  
Tesseract Result:  ia,


----------------------------------------------------------
./data/small/KZE_003.jpg
Fehler Crop bei Bild ./data/small/KZE_003.jpg

----------------------------------------------------------
./data/small/KZE_004.jpg
easyOCR Result:  MASA114 
Tesseract Result:  MA%A 114 |


----------------------------------------------------------
./data/small/KZE_005.jpg
Fehler Crop bei Bild ./data/small/KZE_005.jpg

----------------------------------------------------------
./data/small/KZE_006.jpg
easyOCR Result:  
Tesseract Result:  —


----------------------------------------------------------
./data/small/KZE_007.jpg
easyOCR Result:  
Tesseract Result:  ay


----------------------------------------------------------
./data/small/KZE_008.jpg
easyOCR Result:  MA LI174 
Tesseract Result:  MAeL1174


----------

In [64]:
df[:60]

,Image,Cropping,Text_easyocr,Text_pytesseract,Text_label
0,KZE_001.jpg,Okay,,oe\n,SG GE 567
1,KZE_002.jpg,Okay,,"ia,\n",K BA 4773
2,KZE_003.jpg,Fehler,NaN,NaN,MA OV 7575
3,KZE_004.jpg,Okay,MASA114,MA%A 114 |\n,MA A 114
4,KZE_005.jpg,Fehler,NaN,NaN,MA PP 444E
5,KZE_006.jpg,Okay,,—\n,MA SF 645
6,KZE_007.jpg,Okay,,ay\n,MA H 4000
7,KZE_008.jpg,Okay,MA LI174,MAeL1174\n,MA LI 174
8,KZE_009.jpg,Fehler,NaN,NaN,MA QG 255
9,KZE_010.jpg,Okay,,ae\n,MA ZA 201


In [44]:
df.

85
85
